In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
from component.scripts.gee_parse_reduce_regions import reduceGroups
import ee
from component.scripts.gee import no_remap
from component.scripts.scripts import map_matrix_to_dict
import component.parameter.module_parameter as param
import pygaul

ee.Initialize(project="ee-dfgm2006")
geometries = ee.FeatureCollection(
    [
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [5.262516683862475, 46.39655013792998],
                        [5.262516683862475, 45.63360136660453],
                        [6.580876058862475, 45.63360136660453],
                        [6.580876058862475, 46.39655013792998],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "0"},
        ),
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [9.569157308862474, 47.74335902540055],
                        [9.569157308862474, 46.99933410813134],
                        [10.931461996362474, 46.99933410813134],
                        [10.931461996362474, 47.74335902540055],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "1"},
        ),
        ee.Feature(
            ee.Geometry.Polygon(
                [
                    [
                        [13.084782308862474, 46.6082900232144],
                        [13.084782308862474, 45.87889028991833],
                        [14.315251058862474, 45.87889028991833],
                        [14.315251058862474, 46.6082900232144],
                    ]
                ],
                None,
                False,
            ),
            {"system:index": "2"},
        ),
    ]
)
aoi_name = "Risaralda"
aoi = pygaul.Items(aoi_name)

In [9]:
# Read the default land cover remapping table and convert it to a dictionary

default_remap_matrix = map_matrix_to_dict(param.LC_MAP_MATRIX)

In [48]:
# group1 = ee.Image.random(1).multiply(2).round()
# group2 = ee.Image.random(2).multiply(2).round()
# # group3 = ee.Image.random(3).multiply(2).round()
# image = ee.Image(group1.multiply(3).add(group2))

image = ee.Image.pixelArea().divide(1e6)
group1 = no_remap(ee.Image("users/amitghosh/sdg_module/esa/cci_landcover/2000"), default_remap_matrix)
group2 = ee.Image("users/xavidelamo/SDG1542_Mntn_BioclimaticBelts")


reducer = (
    # ee.Reducer.mean()
    # .combine(ee.Reducer.sum(), None, True)
    ee.Reducer.sum()
        .group(1, "lc")
        .group(2, "biobelt")
        # .group(3, "group2")
)

reduceRegion = (
    image.addBands(group1)
    .addBands(group2)
    # .addBands(group3)
    .reduceRegion(
        **{
            "reducer": reducer,
            "geometry": aoi.geometry(),
            "scale": 1000,
        }
    ).get("groups")
)

reduceRegions = (
    image.addBands(group1)
    .addBands(group2)
    # .addBands(group3)
    .reduceRegions(**{"collection": aoi, "reducer": reducer, "scale": 30})
)

In [49]:
reduceRegionsSum = reduceGroups(
    **{
        "reducer": ee.Reducer.sum(),
        # "reducer": ee.Reducer.sum().forEach(["mean_sum", "sum_sum"]),
        "featureCollection": reduceRegions,
        "groupKeys": ["lc", "biobelt"],
    }
)

In [50]:
reduceRegionsSum.getInfo()

[{'biobelt': 4,
  'groups': [{'lc': 1, 'sum': 0.641873471968588},
   {'lc': 2, 'sum': 109.50770498675655},
   {'lc': 4, 'sum': 102.90517155784703},
   {'lc': 5, 'sum': 3.779320210920271},
   {'lc': 6, 'sum': 0.09795563781738302}]}]

In [51]:
reduceRegion.getInfo()

[{'biobelt': 4,
  'groups': [{'lc': 1, 'sum': 0.9895346875},
   {'lc': 2, 'sum': 108.3664551916666},
   {'lc': 4, 'sum': 104.53144841936276},
   {'lc': 5, 'sum': 2.9568298169117644}]}]

In [ ]:
reducedDict = reduced[0]
acc = ee.Dictionary({})
pathKey = "__path__"

In [ ]:
reducedDict = ee.Dictionary(reducedDict)
acc = ee.Dictionary(acc)
path = reducedDict.getString(pathKey)
groups = path.split("_").map(lambda group: ee.Number.parse(ee.String(group)))

In [ ]:
path.getInfo()

In [ ]:
groups.getInfo()